In [1]:
from visions import *

In [42]:
text = '''O Rose thou art sick. 
The invisible worm, 
That flies in the night 
In the howling storm: 

Has found out thy bed 
Of crimson joy: 
And his dark secret love 
Does thy life destroy.'''

depoeticize(text,
            model_type='bert-base-uncased',
            max_iterations=50,
            randomize=False,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title=None,
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 0.00013018323807045817
o <god> thou art sick. the invisible worm, that flies in the night in the howling storm : has found out thy bed of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 2, score = 0.004045261535793543
o god thou art sick. the invisible worm, that flies in the night in the howling storm : has found out thy <source> of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 3, score = 0.005103247240185738
o god thou art <free>. the invisible worm, that flies in the night in the howling storm : has found out thy source of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 4, score = 0.010653567500412464
o god thou art free. the invisible worm, that flies in the night in the howling storm : has found out thy source of <deepest> joy : and his dark secret love does thy life destroy.
-----------------

'by god thou art blessed. the invisible man, who walks in the night in a hooded cloak : has found both his source of body heat : and his own power that makes his life complete.'

In [2]:
text = '''O Rose thou art sick. 
The invisible worm, 
That flies in the night 
In the howling storm: 

Has found out thy bed 
Of crimson joy: 
And his dark secret love 
Does thy life destroy.'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=50,
            randomize=False,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title=None,
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 4.6452782953565475e-07
o <god> thou art sick. the invisible worm, that flies in the night in the howling storm : has found out thy bed of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 2, score = 7.554813237220515e-07
o god thou art sick. the invisible worm, that flies in the night in the howling storm : has <torn> out thy bed of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 3, score = 0.0002601357700768858
o god thou art <dead>. the invisible worm, that flies in the night in the howling storm : has torn out thy bed of crimson joy : and his dark secret love does thy life destroy.
-----------------------
Iteration 4, score = 0.0002670047106221318
o god thou art dead. the invisible worm, that flies in the night in the howling storm : has torn out thy bed of crimson joy : and <your> dark secret love does thy life destroy.
-----------------------
I

'thank god you are safe. the emotional wind, that blows in the east in a brutal gale : tears leaked out like mud from driving rain : in your eyes only strands of your hair remain.'

In [45]:
text = '''O Rose thou art sick. 
The invisible worm, 
That flies in the night 
In the howling storm: 

Has found out thy bed 
Of crimson joy: 
And his dark secret love 
Does thy life destroy.'''

depoeticize(text,
            model_type='roberta-base',
            max_iterations=50,
            randomize=False,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title=None,
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 9.045047772815451e-05
 O Rose thou< his> sick. The invisible worm, That flies in the night In the howling storm: Has found out thy bed Of crimson joy: And his dark secret love Does thy life destroy.
-----------------------
Iteration 2, score = 9.544840577291325e-05
 O< God> thou his sick. The invisible worm, That flies in the night In the howling storm: Has found out thy bed Of crimson joy: And his dark secret love Does thy life destroy.
-----------------------
Iteration 3, score = 0.0001600701070856303
 O God thou his sick. The invisible< sweet>, That flies in the night In the howling storm: Has found out thy bed Of crimson joy: And his dark secret love Does thy life destroy.
-----------------------
Iteration 4, score = 0.0010359423467889428
 O God thou his sick. The invisible sweet, That flies in the night In the< winter> storm: Has found out thy bed Of crimson joy: And his dark secret love Does thy life destroy.
-----------------------
It

' They are his cold hands. the aluminum shards, his hands in the snow and the melting ice: that carve out his heart from molten clay: and his cold fingers that take his life away.'

In [12]:
text = '''And did those feet in ancient time
Walk upon Englands mountains green:
And was the holy Lamb of God,
On Englands pleasant pastures seen!
 
And did the Countenance Divine,
Shine forth upon our clouded hills?
And was Jerusalem builded here,
Among these dark Satanic Mills?
 
Bring me my Bow of burning gold:
Bring me my arrows of desire:
Bring me my Spear: O clouds unfold!
Bring me my Chariot of fire!
 
I will not cease from Mental Fight,
Nor shall my sword sleep in my hand:
Till we have built Jerusalem,
In Englands green & pleasant Land.'''

depoeticize(text,
            model_type='bert-base-uncased',
            max_iterations=50,
            randomize=False,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            topic=None,
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 8.776559070611256e-07
and did those feet in ancient time walk upon englands mountains green : and was the holy lamb of god, on <our> pleasant pastures seen! and did the countenance divine, shine forth upon our clouded hills? and was jerusalem builded here, among these dark satanic mills? bring me my bow of burning gold : bring me my arrows of desire : bring me my spear : o clouds unfold! bring me my chariot of fire! i will not cease from mental fight, nor shall my sword sleep in my hand : till we have built jerusalem, in englands green & pleasant land.
-----------------------
Iteration 2, score = 1.4563041759174666e-06
and did those feet in ancient time walk upon <our> mountains green : and was the holy lamb of god, on our pleasant pastures seen! and did the countenance divine, shine forth upon our clouded hills? and was jerusalem builded here, among these dark satanic mills? bring me my bow of burning gold : bring me my arrows of desire : b

'and did he who in our time shone upon our pleasant land : and by the holy grace of god, let our pleasant country stand! and did the glorious divine, shine forth upon our pleasant land? and was jerusalem standing here, upon his great almighty hand? bring me my sword of shining light : bring me my weapon of desire : bring me my spear : o lord above! bring me my instrument of fire! i will not die from our fight, nor will my spear be in my hand : for we have reached jerusalem, in our time & pleasant land.'

In [5]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="Strolling through a lovely flower garden",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 2.3689601391652104e-07
tyger tyger, burning bright, in the <stillness> of the night ; what immortal hand or eye, could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 0.0007268207264132798
tyger tyger, burning bright, in the stillness of the night ; what immortal hand or eye, could <stop> thy fearful symmetry?
-----------------------
Iteration 3, score = 1.5245740542013664e-05
tyger tyger, burning bright, in the stillness of the night ; what immortal hand or eye, could stop <such> fearful symmetry?
-----------------------
Iteration 4, score = 1.3895836673327722e-05
tyger tyger, burning bright, in the stillness of the night ; what immortal hand or eye, could stop such fearful <fluttering>?
-----------------------
Iteration 5, score = 0.0009102397598326206
tyger tyger, burning bright, in the stillness of the night ; what <unable> hand or eye, could stop such fearful fluttering?
-----------------------
Iteration 6, scor

'tyger tyger, lovely tree, in a garden by the sea ; what admiring ear or eye, can see such lovely scenery?'

In [8]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="Charging into glorious battle",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 1.6189937014132738e-05
tyger tyger, burning bright, in the forests of the night ; what immortal hand or eye, could frame thy fearful <warrior>?
-----------------------
Iteration 2, score = 6.880984074086882e-06
tyger tyger, burning bright, in the <darkness> of the night ; what immortal hand or eye, could frame thy fearful warrior?
-----------------------
Iteration 3, score = 7.132249265850987e-06
tyger tyger, burning bright, in the darkness of the night ; what immortal hand or eye, could <kill> thy fearful warrior?
-----------------------
Iteration 4, score = 3.1638617201679153e-06
tyger tyger, burning bright, in the darkness of the night ; what immortal hand or eye, could kill <one> fearful warrior?
-----------------------
Iteration 5, score = 0.00034903065534308553
tyger tyger, burning bright, in the darkness of the night ; what immortal hand or eye, could kill one <mighty> warrior?
-----------------------
Iteration 6, score = 0.0144997723

'tyger tyger, shining sword, in the battle stands the lord ; yet unable hand or foot, can fight this mighty warrior?'

In [7]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="The aesthetics of Gothic cathedrals",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 1.6137671536853304e-06
tyger tyger, burning bright, in the <darkness> of the night ; what immortal hand or eye, could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 0.003593442030251026
tyger tyger, burning bright, in the darkness of the night ; what immortal hand or eye, could frame <such> fearful symmetry?
-----------------------
Iteration 3, score = 3.435142934904434e-05
tyger tyger, burning bright, in the darkness of the night ; what immortal hand or eye, could frame such fearful <images>?
-----------------------
Iteration 4, score = 0.00021723529789596796
tyger tyger, burning bright, in the darkness of the night ; what <illusion> hand or eye, could frame such fearful images?
-----------------------
Iteration 5, score = 0.001161798369139433
tyger tyger, burning bright, in the darkness of the night ; what illusion hand or eye, could frame such <striking> images?
-----------------------
Iteration 6, score = 0.0006

'tyger tyger, gothic spire, in the middle of the shire ; which cathedral bore that spire, which bore that gothic pinnacle?'

In [9]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="Urban planning and traffic management",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 5.617350780084962e-07
tyger tyger, burning bright, in the <middle> of the night ; what immortal hand or eye, could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 0.00030281703220680356
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could <stop> thy fearful symmetry?
-----------------------
Iteration 3, score = 0.0002120196004398167
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could stop <such> fearful symmetry?
-----------------------
Iteration 4, score = 5.888508167117834e-05
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could stop such fearful <energy>?
-----------------------
Iteration 5, score = 0.00010078400373458862
tyger tyger, burning bright, in the middle of the night ; what <unable> hand or eye, could stop such fearful energy?
-----------------------
Iteration 6, score = 0.002400835277512669

'tyger tyger, traffic light, in the middle of the night ; does policeman check his lights, not check his traffic cameras?'

In [11]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="Leaves of Grass",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 3.2061423098639352e-06
tyger tyger, burning bright, in the <silence> of the night ; what immortal hand or eye, could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 0.00027151184622198343
tyger tyger, burning bright, in the silence of the night ; what immortal hand or eye, could frame thy fearful <messenger>?
-----------------------
Iteration 3, score = 1.4436841411225032e-06
tyger tyger, burning bright, in the silence of the night ; what immortal hand or eye, could <stop> thy fearful messenger?
-----------------------
Iteration 4, score = 5.6535522162448615e-05
tyger tyger, burning bright, in the silence of the night ; what <unable> hand or eye, could stop thy fearful messenger?
-----------------------
Iteration 5, score = 0.001291179214604199
tyger tyger, burning bright, in the silence of the night ; what unable hand or eye, could stop <this> fearful messenger?
-----------------------
Iteration 6, score = 0.0028667

'tyger tyger, broken glass, in the middle of the grass ; leaves forever blown by wind, how much more petty agony?'

In [4]:
text = '''{Tyger Tyger}, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_path='pofo',
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="A little, fluffy kitty cat",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 1.7358479453832842e-06
tyger tyger, burning bright, in the <middle> of the night ; what immortal hand or eye, could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 0.00043767152237705886
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could frame <such> fearful symmetry?
-----------------------
Iteration 3, score = 3.0236702514230274e-05
tyger tyger, burning bright, in the middle of the night ; what <suspicious> hand or eye, could frame such fearful symmetry?
-----------------------
Iteration 4, score = 0.00016313124797306955
tyger tyger, burning bright, in the middle of the night ; what suspicious hand or eye, could <cause> such fearful symmetry?
-----------------------
Iteration 5, score = 4.726382758235559e-05
tyger tyger, <blinking> bright, in the middle of the night ; what suspicious hand or eye, could cause such fearful symmetry?
-----------------------
Iteration 6, score = 

'tyger tyger, kitty cat, in the middle of a chat ; what electric freak was he, to send such nasty messages?'

In [167]:
text = '''[Tyger Tyger], burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_type='roberta-base',
            max_iterations=100,
            randomize=0.0,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            topic="flowers",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 2.446381097698236e-09
 Tyger Tyger,< Standing> bright, In the forests of the night; What immortal hand or eye, Could frame thy fearful symmetry?
-----------------------
Iteration 2, score = 5.7247134463978e-06
 Tyger Tyger, Standing< high>, In the forests of the< sky>; What immortal hand or eye, Could frame thy fearful symmetry?
-----------------------
Iteration 3, score = 3.0911166959413094e-06
 Tyger Tyger, Standing high, In the< centre> of the sky; What immortal hand or eye, Could frame thy fearful symmetry?
-----------------------
Iteration 4, score = 2.5605545488360804e-06
 Tyger Tyger, Standing high, In the centre of the sky; What immortal hand or eye, Could frame< you> fearful symmetry?
-----------------------
Iteration 5, score = 0.00012871516810264438
 Tyger Tyger, Standing high, In the centre of the sky; What immortal hand or eye, Could< give> you fearful symmetry?
-----------------------
Iteration 6, score = 6.3166094150801655e-06

' Tyger Tyger, Flowers lay, In the middle of the day; Are poetic words just words, To give you extra energy?'

In [26]:
text = '''Shall I compare thee to a summer's day? 
Thou art more lovely and more fascinate:
Rough winds do shake the darling buds of May,
And summer's lease hath all too short a date: '''

depoeticize(text,
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.0,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            topic="flowers",
            strong_topic_bias=2.0,
            sequential=True,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 16, score = 0.003925323020666838
shall i compare thee to a summer's day? thou art more lovely <the> more fascinate : rough winds do shake the darling buds of may, and summer's lease hath all too short a date :
-----------------------
Iteration 18, score = 0.0008427134682307269
shall i compare thee to a summer's day? thou art more lovely the more <evergreen> : rough winds do shake the darling buds of may, and summer's lease hath all too short a <green> :
-----------------------
Iteration 20, score = 0.07224021852016449
shall i compare thee to a summer's day? thou art more lovely the more evergreen : <brisk> winds do shake the darling buds of may, and summer's lease hath all too short a green :
-----------------------
Iteration 25, score = 0.7040115594863892
shall i compare thee to a summer's day? thou art more lovely the more evergreen : brisk winds do shake the <berry> buds of may, and summer's lease hath all too short a green :
-----------------------

"shall i compare thee to a summer's day? thou art more lovely the more evergreen : brisk winds do shake the berry leaves of may, a summer lease hath all too short a green :"

In [11]:
text = '''Tyger Tyger, burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

depoeticize(text,
            model_type='bert-base-uncased',
            max_iterations=100,
            randomize=0.1,
            cooldown=0.05,
            stop_score=1.0,
            match_meter=True,
            match_rhyme=True,
            preserve_punctuation=True,
            title="Computational language modeling with artificial neural networks",
            strong_topic_bias=2.0,
            modifier=None)#metalness_modifier())

-----------------------
Iteration 1, score = 1.6370143088306754e-11
tyger tyger, burning bright, in the forests of the night ; what immortal hand or eye, could frame <create> fearful symmetry?
-----------------------
Iteration 2, score = 2.7015327752621943e-09
tyger tyger, burning bright, in the forests of the night ; what immortal hand or eye, could <you> create fearful symmetry?
-----------------------
Iteration 3, score = 2.5341921627841657e-07
tyger tyger, burning bright, in the <middle> of the night ; what immortal hand or eye, could you create fearful symmetry?
-----------------------
Iteration 4, score = 4.249602625350235e-07
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could you create <such> symmetry?
-----------------------
Iteration 5, score = 5.688901794087542e-09
tyger tyger, burning bright, in the middle of the night ; what immortal hand or eye, could you create such <things>?
-----------------------
Iteration 6, score = 1.182900604

'this poem is about noticing that in the middle of the poem exists a computational neural network. how can humans do such things :'

In [151]:
text = '''[Tyger Tyger], burning bright, 
In the forests of the night; 
What immortal hand or eye, 
Could frame thy fearful symmetry?'''

parody(text,
       model='gpt2',
       match_meter=True,
       match_rhyme=True,
       title="a little, fluffy kitty cat",
       randomize=0.005,
       modifier=None)

' Tyger Tyger, also known in the English as "the lone wolf," created this cat, named "T-Rex," by writing poetry'

In [4]:
print(bouts_rimés('cloud hills crowd daffodils trees breeze'.split(' '),
                  model_type='bert-base-uncased',
                  model_path='pofo',
                  meter='u-u-u-u-',
                  title='I Wandered Lonely as a Cloud',
                  author='William Wordsworth',
                  strong_topic_bias=None))

opponents simulator {cloud},
the education rocked a {hills}.
recapture to a hui {crowd},
royale disposed the {daffodils}.
a vickers macy shrines and {trees},
and overseeing of a {breeze}.
-----------------------
Iteration 1, score = 2.7611060104959506e-08
opponents <susquehanna> cloud, the education rocked a hills. recapture to a hui crowd, royale disposed the daffodils. a vickers macy shrines and trees, and overseeing of a breeze.
-----------------------
Iteration 2, score = 8.648763838436935e-08
opponents susquehanna cloud, the education rocked a hills. recapture to a hui crowd, royale disposed the daffodils. a <city> macy shrines and trees, and overseeing of a breeze.
-----------------------
Iteration 3, score = 1.334836667865602e-07
opponents susquehanna cloud, the education rocked a hills. recapture to a hui crowd, <alone> disposed the daffodils. a city macy shrines and trees, and overseeing of a breeze.
-----------------------
Iteration 4, score = 1.0238689895913922e-07
opponents

In [7]:
print(bouts_rimés('storm form'.split(' '),
                  model_type='bert-base-uncased',
                  model_path='pofo',
                  meter='u-u-u-u-u-',
                  title=None,
                  strong_topic_bias=None))

requests symmetric darren soccer {storm},
the toad italics for croatian {form}.
-----------------------
Iteration 1, score = 4.584346243063919e-06
requests symmetric darren soccer storm, the <first> italics for croatian form.
-----------------------
Iteration 2, score = 2.7776372917287517e-06
requests symmetric darren soccer storm, the first <submission> for croatian form.
-----------------------
Iteration 3, score = 1.1476559848233592e-05
requests <included> darren soccer storm, the first submission for croatian form.
-----------------------
Iteration 4, score = 0.00042704015504568815
requests included darren soccer storm, the first submission for <another> form.
-----------------------
Iteration 5, score = 5.099064583191648e-05
requests included <project> soccer storm, the first submission for another form.
-----------------------
Iteration 6, score = 0.0009247412672266364
requests included project <solar> storm, the first submission for another form.
-----------------------
Iteratio